# Track Management of a Digital Music Store

## Database Description
This dataset contains the information about a fictional digital music store, Chinook. The original dataset can be downloaded here *[Chinook Database](https://github.com/lerocha/chinook-database)*.  
   
This database contains rich information on the employee, customers and the details on the customers' purchases such as the expenses, tracks & albums purchased.
  
## Schema Diagram
Table names are in bold; primary keys are shaded.
![](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg)

## Data Overview  

In [55]:
%%html
<style>
table {float:left}
</style>

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook_ori.db

In [3]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook_ori.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Selecting New Albums to Stock

Our store (*Chinook*) just signed a deal with a new record label and they provided four albums for us to choose from. We need to select three of them to start with. The artist names and the genre of music they produce are listed as below. All four artists do not have any tracks in the store yet. 
  
We are going to find out the bestselling genres in the store. The top three popular genres in the table below will be selected. Since the record label focuses on the US market, we will limit the query in the US customers.


| Artist Name | Genre |
| ---: | ---: |
| Regal | Hip-Hop |
| Red Ton | Punk |
| Meteor and the Girls | Pop |
| Slim Jim Bites | Blues |  


In [115]:
%%sql
WITH tracks_sold_usa AS 
    (
     SELECT il.* FROM invoice_line il
      INNER JOIN invoice i ON i.invoice_id = il.invoice_id
      INNER JOIN customer c ON c.customer_id = i.customer_id
      WHERE c.country == 'USA'
    )
    
SELECT 
    g.name Genre,
    COUNT(g.genre_id) Quantity_Sold,
    ROUND(CAST(COUNT(g.genre_id) AS Float) / CAST((SELECT COUNT(*) FROM tracks_sold_usa) AS Float), 3) PCT
  FROM tracks_sold_usa tsu
 INNER JOIN track t ON tsu.track_id = t.track_id
 INNER JOIN genre g ON g.genre_id = t.genre_id
 GROUP BY g.genre_id
 ORDER BY PCT DESC
 LIMIT 10;

 * sqlite:///chinook_ori.db
Done.


Genre,Quantity_Sold,PCT
Rock,561,0.534
Alternative & Punk,130,0.124
Metal,124,0.118
R&B/Soul,53,0.05
Blues,36,0.034
Alternative,35,0.033
Pop,22,0.021
Latin,22,0.021
Hip Hop/Rap,20,0.019
Jazz,14,0.013


Based on this data, we select the three albums below:

| Artist Name | Genre |
| ---: | ---: |
| Red Ton | Punk |
| Meteor and the Girls | Pop |
| Slim Jim Bites | Blues |  

## Analying the Sales by Country

It is essential to look into the sales revenue in different markets. We now take a look at these aspects of sales in each country.

- Number of customers
- Total sales
- Average order value
- Average value of sales per customer

For those countries with only one customer, we will group these customers as **Other** in the analysis.

In [6]:
%%sql
WITH sales_by_country AS 
   (
    SELECT 
        CASE
            WHEN (
                SELECT COUNT(*)
                  FROM customer
                 WHERE country = c.country
                ) = 1 THEN 'Other'
            ELSE
                c.country
        END Country,
        ROUND(SUM(i.total), 3) Total_Sales,
        COUNT(distinct c.customer_id) Customer_Number,
        ROUND(SUM(i.total) / CAST(COUNT(distinct i.customer_id) AS Float), 3) AVG_Customer_Value,
        ROUND(SUM(i.total) / CAST(COUNT(i.invoice_id) AS Float), 3) AVG_Order_Value
      FROM customer c
      INNER JOIN invoice i ON c.customer_id = i.customer_id
      GROUP BY 1
      ORDER BY Total_Sales DESC
   )

SELECT 
    Country,
    Total_Sales,
    Customer_Number,
    AVG_Customer_Value,
    AVG_Order_Value
  FROM 
    (SELECT
        sc.*,
        CASE
            WHEN sc.Country = 'Other' THEN 1
            ELSE 0
        END sort
       FROM sales_by_country sc
       GROUP BY sc.Country
       ORDER BY sort ASC, sc.Total_Sales DESC
    )

 * sqlite:///chinook_ori.db
Done.


Country,Total_Sales,Customer_Number,AVG_Customer_Value,AVG_Order_Value
USA,1040.49,13,80.038,7.943
Canada,535.59,8,66.949,7.047
Brazil,427.68,5,85.536,7.011
France,389.07,5,77.814,7.781
Germany,334.62,4,83.655,8.161
Czech Republic,273.24,2,136.62,9.108
United Kingdom,245.52,3,81.84,8.769
Portugal,185.13,2,92.565,6.384
India,183.15,2,91.575,8.721
Other,1094.94,15,72.996,7.449


Regarding to the *total sales*, **the US** is the largest market of Chinook, followed by **Canada** and **Brazil**. However, we notice that the *average customer value* of **Czech** is a lot higher than other countries, by at least \$55.945. The customers from Czech also spent the most on each order on average. Other two countries with relatively high *average customer value* are **Portugal** and **India**, both are above $90. This view indicates a great potential of these markets.
  
The special group **Other** that includes all countries with only one customer contributes more than the largest single market the US.
  
## Investigate the Purchasing Methods: Albums vs Individual Tracks¶

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:
- purchase a whole album
- purchase a collection of one or more individual tracks.  

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Managers are currently considering changing our purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We now will find out **what percentages of purchases are individual tracks vs whole albums**. It helps the managers understand what effect this decision might have on the overall revenue.

In this instance, there are two edge cases to consider:
- Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.
- Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.
In the first case, since our analysis is concerned with maximizing revenue we can safely ignore albums consisting of only a few tracks. The company has previously done analysis to confirm that the second case does not happen often, so we can ignore this case also.

In [116]:
%%sql
WITH track_album AS
    (
    SELECT t.track_id,
           a.album_id
      FROM track t
     INNER JOIN album a ON a.album_id = t.album_id
    ),

    track_invoice_album AS
    (
    SELECT ta.track_id,
           ta.album_id,
           il.invoice_id
      FROM invoice_line il
     INNER JOIN track_album ta ON il.track_id = ta.track_id
    ),

    all_purchase AS
    (
    SELECT 
        tia.invoice_id,
        CASE
            WHEN
                (
                SELECT track_id FROM track_album ta WHERE ta.album_id = tia.album_id
                EXCEPT
                SELECT track_id FROM track_invoice_album tia2 WHERE tia2.invoice_id = tia.invoice_id
                ) IS NULL
                AND
                (
                SELECT track_id FROM track_invoice_album tia2 WHERE tia2.invoice_id = tia.invoice_id  
                EXCEPT
                SELECT track_id FROM track_album ta WHERE ta.album_id = tia.album_id
                ) IS NULL
            THEN 1
            ELSE 0
        END is_album
      FROM track_invoice_album tia
    ),
    
    is_album AS
    (
    SELECT
        invoice_id,
        MAX(is_album) as is_album
      FROM all_purchase
     GROUP BY invoice_id
    )
    

SELECT
    SUM(is_album) Album_Purchase,
    COUNT(*) - SUM(is_album) Track_Purchase,
    ROUND(CAST(SUM(is_album) AS FLOAT) / CAST(COUNT(*) AS FLOAT), 3)*100 PCT_Album_Purchase
  FROM is_album

 * sqlite:///chinook_ori.db
Done.


Album_Purchase,Track_Purchase,PCT_Album_Purchase
114,500,18.6


It is clear that the majority of purchases were done by selecting individual tracks rather than just picking the whole album. I would suggest the managers not to change our selling strategy, because then we will probably upset almost 20% of the customers.
  
## How many tracks in the stores have never been purchased, and which genres do they belong to?
To optimize our stock stratege, next we would like to know if we have purchased many tracks that are not popular enough (never been selected by the customers).

In [117]:
%%sql
WITH track_genre_invoice AS
    (
    SELECT 
        t.track_id track_id,
        g.name genre,
        il.invoice_id invoice_id
      FROM track t
      LEFT JOIN invoice_line il ON t.track_id = il.track_id 
      INNER JOIN genre g ON g.genre_id = t.genre_id
    ),

    is_purchased AS
    (
    SELECT 
        genre,
        track_id,
        CASE
            WHEN
                (
                SELECT tgi2.invoice_id
                  FROM track_genre_invoice tgi2 
                 WHERE tgi2.invoice_id = tgi.invoice_id
                   AND tgi2.track_id = tgi.track_id
                ) IS NULL 
            THEN 1
            ELSE 0
        END unpurchased
    FROM track_genre_invoice tgi
    )
 
SELECT 
 genre Genre,
 COUNT(distinct track_id) Total_Track_Num,
 SUM(unpurchased) Unpurchased_Num,
 ROUND(CAST(SUM(unpurchased) AS FLOAT) / CAST(COUNT(distinct track_id) AS FLOAT), 3) PCT_Unpurchased
  FROM is_purchased
 GROUP BY genre
 ORDER BY PCT_Unpurchased DESC

 * sqlite:///chinook_ori.db
Done.


Genre,Total_Track_Num,Unpurchased_Num,PCT_Unpurchased
World,28,28,1.0
Science Fiction,13,13,1.0
Sci Fi & Fantasy,26,26,1.0
Rock And Roll,12,12,1.0
Opera,1,1,1.0
Comedy,17,17,1.0
Bossa Nova,15,15,1.0
Drama,64,63,0.984
TV Shows,93,91,0.978
Soundtrack,43,38,0.884


There are tracks of several genres are not sold at all, such as **World**, **Science Fiction**, **SCI Fi & Fanctasy**, **Rock and Roll**, **Opera**, **Comedy** and **Bosa Nova**. **Drama** and **TV Shows** only sold 1 or 2 tracks too. We should note that the genre **Opera** is differnt from others because only 1 track is on the shelf, it is therefore easy to have 100% unpurchased rate.  
  
Some of the less-sold genres share some common features, such as **Science Fiction** and **SCI Fi & Fanctasy**, **Opera** and **Drama**, **Comedy** and **TV Shows**, we can probably make them a duo and do some sales promotions, or hold some introduction sessions to improve their polularity. 
  
It it worth noting that we have 579 tracks in stock for **Latin** music, indicating a great range of Latin tracks we have here in Chinook. Nevertheless 79.4% of them have never been purchased. We should further look into this genre to find out the issue.
